In [61]:
%matplotlib inline
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import os
import datetime
from os.path import join
from nilearn.masking import apply_mask
from nilearn.image import threshold_img,math_img,mean_img
from nilearn import plotting, input_data, image
from scipy.stats import ttest_ind
from scipy.ndimage import morphology,measurements
from nibabel import Nifti1Image

rsn10 = r'C:\github\a_htywork\20170315_autumnROC\PNAS_Smith09_rsn10.nii.gz'
face_dir = r'C:\expdata\ABIDE_2017_drz5000\FACE_drzstat_N1000'
abide_mask_dir = r'C:\expdata\ABIDE_2017_drz5000\ABIDE12_drz5000_mask'
result_dir1 = r'C:\temp\all3'

def safe_mkdir(dirname):
    try:
        os.mkdir(dirname)
        
    except:
        print('mkdir failed!')


def getAuc_P(drz_ff,mask,target):
    try:
        roivalue = np.mean(apply_mask(drz_ff, mask),1)
        fpr, tpr, thresholds = metrics.roc_curve(target, roivalue)
        auc = metrics.auc(fpr, tpr)
        if auc < 0.5:
            fpr, tpr, thresholds = metrics.roc_curve(target, -roivalue)
            auc = metrics.auc(fpr, tpr)
            
        t, p = ttest_ind(roivalue[(target==0).nonzero()[0]],
                             roivalue[(target==1).nonzero()[0]],
                             equal_var=True)

    except BaseException as e:
        roivalue = target*0        
        auc = 0
        t = 0
        p = 1
        
    return roivalue.reshape(-1,1), auc, t, p

def gpstr(p):
    pstr = '%.3f' % p
    if p < 0.05:pstr += '*'
    if p < 0.01:pstr += '*'
    return pstr


def getcluster(abide_mask,t1t2,iternum=1):
    data = morphology.binary_opening(abide_mask.get_data(), iterations=iternum)
    #data = morphology.binary_dilation(data, iterations=1)
    data,labelnum = measurements.label(data)

    if t1t2 =='t1':
        fstr = str(ii+1)
    else:
        fstr = str(ii+11)

    for jjj in range(1,labelnum+1):
        cluster = (data==jjj)
        abide_mask = Nifti1Image(cluster, header=abide_mask.header, affine=abide_mask.affine)
        roivalueA1, aucA1, tA1, pA1 = getAuc_P(facedata_ff,abide_mask,face_target)
        maskt1pxs = abide_mask.get_data().sum()
        if pA1 < 0.05:
            math_img('img1*1.0',img1=abide_mask).to_filename(join(result_dir,'A%smask_cluster%d.nii.gz' % (fstr,jjj)))
            plotting.plot_roi(abide_mask,title='A%s:z%d,%.3f' % (fstr,zthr,pA1), draw_cross=False,
                          output_file = join(result_dir,'A%s_z%s_cluster%d.jpg' % (fstr,zstr,jjj)))
            print('A%s: cluster%d (%d):%.3f(%s)' % \
                  (ii+1,fstr,jjj,maskt1pxs,aucA1,gpstr(pA1)))
    return labelnum
            
def nii_opening(abide_mask,iternum=1):
    data = morphology.binary_opening(abide_mask.get_data(), iterations=iternum)
    abide_mask = Nifti1Image(data, header=abide_mask.header, affine=abide_mask.affine)
    return abide_mask

In [64]:
result_dir = join(result_dir1,datetime.datetime.now().strftime("Mask%m%d_%H%M%S"))
safe_mkdir(result_dir)

face_target = np.ones((43,1))
face_target[:23] = 0
zthr = 4
zstr = str(zthr).replace('.','p')
RSNstr = ['PVN','OPN','LVN','DMN','CEB','MOT','AUD','ECN','RFP','LFP']
opening = 1
'''
with open(join(abide_mask_dir,'subjdx.txt'), 'r') as f:
    abide_target = [int(line.strip()) for line in f]
'''
RSN_report =''
for ii in range(10):
    y = face_target
    facedata_ff = join(face_dir,'rsn%d' % ii,'allRmaps.nii.gz')
    abide_ff = join(abide_mask_dir,'rsn%d' % ii,'allRmaps.nii.gz')
    
    # get RSN mask with Z threshold
    rsnmask =  math_img('img1>=%f' % zthr,img1=image.index_img(rsn10, ii))
    
        
    abide_maskt1_ff =join(abide_mask_dir,'rsn%d' % ii,'twosample_tfce_corrp_tstat1.nii.gz')
    abide_maskt2_ff =join(abide_mask_dir,'rsn%d' % ii,'twosample_tfce_corrp_tstat2.nii.gz')

    abide_maskt1_ff = math_img('img1>=%f' % 0.95,img1=abide_maskt1_ff)
    abide_maskt2_ff = math_img('img1>=%f' % 0.95,img1=abide_maskt2_ff)
    
    abide_maskt1_ff = math_img('img1*img2',img1=rsnmask, img2=abide_maskt1_ff)  
    abide_maskt2_ff = math_img('img1*img2',img1=rsnmask, img2=abide_maskt2_ff)
    

    
    

    
    roivalueR, aucR, tR, pR = getAuc_P(facedata_ff,rsnmask,face_target)
    
    
    # perform opening
    #abide_maskt1_ff = abide_maskt2_ff
    
        

    getcluster(abide_maskt1_ff,'t1',iternum=opening)
    getcluster(abide_maskt2_ff,'t2',iternum=opening)
    
    abide_maskt1_ff = nii_opening(abide_maskt1_ff,iternum=opening)
    abide_maskt2_ff = nii_opening(abide_maskt2_ff,iternum=opening)
    
    
    math_img('img1*1.0',img1=abide_maskt1_ff).to_filename(join(result_dir,'A%02dmask.nii.gz' % (ii+1)))
    math_img('img1*1.0',img1=abide_maskt2_ff).to_filename(join(result_dir,'A%02dmask.nii.gz' % (ii+11)))

    
    roivalueR, aucR, tR, pR = getAuc_P(facedata_ff,rsnmask,face_target)
    roivalueA1, aucA1, tA1, pA1 = getAuc_P(facedata_ff,abide_maskt1_ff,face_target)
    roivalueA2, aucA2, tA2, pA2 = getAuc_P(facedata_ff,abide_maskt2_ff,face_target)
    
    plotting.plot_roi(abide_maskt1_ff,title='RSN%d:t1,%d,%.3f' % (ii+1,zthr,pA1),
                      output_file = join(result_dir,'RSN%d_t1_%s.jpg' % (ii+1,zstr)))
    plotting.plot_roi(abide_maskt2_ff,title='RSN%d:t2,%d,%.3f' % (ii+1,zthr,pA2),
                      output_file = join(result_dir,'RSN%d_t2_%s.jpg' % (ii+1,zstr)))

    
    

    
    maskt1pxs = abide_maskt1_ff.get_data().sum()
    maskt2pxs = abide_maskt2_ff.get_data().sum()

    RSN_report += ('RSN%02d_%s AUC: %.3f(%s)\t t1(%d):%.3f(%s)\t\t t2(%d):%.3f(%s) \n' % \
          (ii+1,RSNstr[ii],aucR,gpstr(pR),
           maskt1pxs,aucA1,gpstr(pA1),
           maskt2pxs,aucA2,gpstr(pA2)))



    if ii == 0:
        Rmatrix = roivalueR
        A1matrix = roivalueA1
        A2matrix = roivalueA2
    else:
        Rmatrix = np.hstack((Rmatrix,roivalueR))
        A1matrix = np.hstack((A1matrix,roivalueA1))
        A2matrix = np.hstack((A2matrix,roivalueA2))
    

print(RSN_report)

np.savetxt(join(result_dir,'tsall_%s.csv' % zstr),np.hstack((face_target,Rmatrix,A1matrix,A2matrix)),fmt='%.5f', delimiter=',')

C:\Anaconda3\lib\site-packages\numpy\ma\core.py:4185: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


RSN03_LVN: t1 cluster1 (2796):0.672(0.029*)
RSN03_LVN: t2 cluster1 (29):0.652(0.049*)
RSN04_DMN: t1 cluster3 (1401):0.739(0.007**)
RSN05_CEB: t2 cluster1 (1532):0.630(0.034*)
RSN08_ECN: t2 cluster1 (4763):0.661(0.031*)
RSN09_RFP: t2 cluster1 (306):0.670(0.050*)
RSN09_RFP: t2 cluster6 (7):0.689(0.049*)
RSN10_LFP: t1 cluster2 (50):0.707(0.040*)
RSN01_PVN AUC: 0.533(0.875)	 t1(2004):0.509(0.903)		 t2(61):0.561(0.302) 
RSN02_OPN AUC: 0.583(0.637)	 t1(0):0.000(1.000)		 t2(0):0.000(1.000) 
RSN03_LVN AUC: 0.593(0.176)	 t1(4677):0.607(0.143)		 t2(29):0.652(0.049*) 
RSN04_DMN AUC: 0.609(0.407)	 t1(4029):0.759(0.010**)		 t2(0):0.000(1.000) 
RSN05_CEB AUC: 0.567(0.091)	 t1(0):0.000(1.000)		 t2(1571):0.626(0.035*) 
RSN06_MOT AUC: 0.565(0.313)	 t1(7):0.620(0.198)		 t2(2373):0.570(0.239) 
RSN07_AUD AUC: 0.628(0.124)	 t1(7):0.678(0.101)		 t2(36):0.557(0.369) 
RSN08_ECN AUC: 0.630(0.131)	 t1(1129):0.604(0.209)		 t2(4849):0.661(0.032*) 
RSN09_RFP AUC: 0.622(0.041*)	 t1(71):0.524(0.789)		 t2(795):0.657(